<a href="https://colab.research.google.com/github/TAHKInteractingAI/Connect_Linkedin_Colab/blob/main/Test_Sheet_Connect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread oauth2client
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
import pandas as pd
from google.colab import auth
from google.auth import default
from googleapiclient.discovery import build

In [ ]:
from selenium import webdriver

def web_driver():

    options = webdriver.ChromeOptions()

    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()

In [ ]:
import os
import sys
import time
from selenium.webdriver.support import expected_conditions as EC

def Login_Linkedin(driver):

    baseDir = os.path.dirname(os.path.realpath(sys.argv[0])) + os.path.sep
    """ Setup Argument Parameters """
    #config = configparser.RawConfigParser()
    #config.read('/content/sample_data/Account.cfg')

    #username = config.get('CREDS', 'linkedin_username')
    username = 'Henry.Universes@TAHKfoundation.org'
    #password = config.get('CREDS', 'linkedin_password')
    password = '2024@Thanhddx'
    # head to  login page
    driver.get("https://www.linkedin.com/home")
    time.sleep(10)
    # Trường hợp 1: Có nút "Sigin with email"
    try:
      email_signin_button = driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/div/a')
      email_signin_button.click()
      driver.find_element(By.XPATH, '//*[@id="username"]').send_keys(username)
      driver.find_element(By.XPATH, '//*[@id="password"]').send_keys(password)
      signin_button = driver.find_element(By.XPATH, '//*[@id="organic-div"]/form/div[3]/button')
      signin_button.click()
      print("Đã nhấn nút đăng nhập")
    # Trường hợp 2: Không có nút "Sigin with email"
    except:
      driver.find_element(By.XPATH, '//*[@id="session_key"]').send_keys(username)
      driver.find_element(By.XPATH, '//*[@id="session_password"]').send_keys(password)
      signin_button = driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/div/form/div[2]/button')
      signin_button.click()
      print("Đã nhấn nút đăng nhập")

    #Nếu bắt nhập mã verify
    try:
      code_input = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/main/form/div[1]/input[15]')))
      code = input("Nhập mã code vừa gửi email: ")
      code_input.send_keys(code)
      submit_button = driver.find_element(By.XPATH, '/html/body/div/main/form/div[2]/button')
      submit_button.click()
    except:
      print("Login không thành công")

    time.sleep(5)

In [ ]:
def autoConnect(driver):
  # Authenticate
  auth.authenticate_user()
  creds, _ = default()

  # Create the service
  service = build('sheets', 'v4', credentials=creds)

  #ID sheet Connect Linkedin
  SPREADSHEET_ID = '1QI2SPIwBgUJam8DwQXt-JbcIRlOtaqgLHNXtVGY1Z4A'

  # Range of cells we want to retrieve
  RANGE_NAME = 'Sheet1!A:D'  # Explicitly specify the range to avoid including extra columns

  # Call the Sheets API
  sheet = service.spreadsheets()
  result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
  values = result.get('values', [])
  slen = len(values) + 1

  for row in range(2, slen):
    link = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!A{row}').execute()
    driver.get(link['values'][0][0])
    time.sleep(6)
    try:
            followButon = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/button')
            button_text1 = followButon.get_attribute("aria-label")

            if "Invite" in button_text1:
                try:
                    print("This person is not connected , try to click connect..........")
                    followButon.click()
                    print("click connect!")
                    time.sleep(2)

                    connectButonNew = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[3]/button[2]")
                    connectButonNew.click() # Nhấn nút send without a note
                    time.sleep(2)

                    checkStatus = followButon.get_attribute("aria-label")
                    if "Pending" in checkStatus:
                        print("Invite sent !" + link['values'][0][0])
                        sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!B{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Pending"]]}).execute()

                        sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!C{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Invite Sent"]]}).execute()

                        sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!D{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Message not sent"]]}).execute()

                    else:
                        print("Not connected one !" + link['values'][0][0])
                        sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!B{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Not Connected"]]}).execute()

                        sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!C{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Not Connected"]]}).execute()

                        sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!D{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Message not sent"]]}).execute()
                except NoSuchElementException:
                    print("Not connected one !" + link['values'][0][0])
                    sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!B{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Not Connected"]]}).execute()

                    sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!C{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Not Connected"]]}).execute()

                    sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!D{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Message not sent"]]}).execute()
            elif "Follow" in button_text1:
                try:
                    connectButonNew1 = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/button')
                    connectButonNew1.click() # Tìm và nhấn nút More
                    time.sleep(1)

                    connectButonNew2 = driver.find_element(By.XPATH,"/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/div/div/ul/li[3]/div")
                    connectButonNew2.click() # Tìm nút connect

                    connectButonNew3 = driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div[3]/button[2]")
                    connectButonNew3.click() # Nhấn send without a note
                    time.sleep(2)

                    followButon1 = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/button")
                    test = followButon1.get_attribute("aria-label")
                    if("Following" in test):
                        sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!B{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Not Connected"]]}).execute()

                        sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!C{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Invite sent"]]}).execute()

                        sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!D{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Message not sent"]]}).execute()

                    else:
                        sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!B{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Not Connected"]]}).execute()

                        sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!C{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Not Connected"]]}).execute()

                        sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!D{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Message not sent"]]}).execute()
                except NoSuchElementException:
                    print("Not connected one !" + link['values'][0][0])
                    sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!B{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Not Connected"]]}).execute()

                    sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!C{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Not Connected"]]}).execute()

                    sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!D{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Message not sent"]]}).execute()
            elif "Pending" in button_text1:
                    print("Pending!" + link['values'][0][0])
                    sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!B{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Pending"]]}).execute()

                    sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!C{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Pending"]]}).execute()

                    sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!D{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Message not sent"]]}).execute()
                    print("write in csv!")
    except NoSuchElementException:
            try:
                checkStatus = driver.find_element(By.XPATH,
                                                  "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/button")
                checkStatus.click()
                time.sleep(1)
                checkStatus = driver.find_element(By.XPATH,
                                                  "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/div/div/ul/li[7]/div")
                checkStatus2 = checkStatus.get_attribute("aria-label")
                if "Remove" in checkStatus2:
                    print("Connected one!" + link['values'][0][0])
                    sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!B{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Connected"]]}).execute()

                    sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!C{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Connected one!"]]}).execute()

                    sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!D{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Message not sent"]]}).execute()
                    print("write in csv!")
                    try:
                        sendMessageButton = driver.find_element(By.XPATH,"/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[1]/button")
                        print("Message button found")
                        sendMessageButton.click()
                        time.sleep(1)

                        driver.find_element(By.XPATH,"/html/body/div[5]/div[4]/aside[1]/div[2]/div[1]/div[2]/div/div[3]/div[3]/form/div[3]/div/div/div[1]").send_keys('Message')

                        time.sleep(1)
                        driver.find_element(By.XPATH,
                                            "/html/body/div[5]/div[4]/aside[1]/div[2]/div[1]/div[2]/div/div[3]/div[3]/form/footer/div[2]/div/button").click()
                        time.sleep(4)
                        print("click")
                        sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!B{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Connected"]]}).execute()

                        sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!C{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Connected one!"]]}).execute()

                        sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!D{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Message sent"]]}).execute()
                    except NoSuchElementException:
                        try:
                            # button_text2 = driver.find_element(By.XPATH,"/html/body/div[5]/div[4]/aside[1]/div[3]/div[1]/div[2]/div/form/div[3]/div/div[1]/div[1]/p")
                            # time.sleep(2)
                            driver.find_element(By.XPATH, "//div[contains(@class, 'msg-form__contenteditable')]").send_keys('Message')
                            time.sleep(2)
                            # button_text2.send_keys(data['Message'][value])
                            driver.find_element(By.XPATH,
                                                "/html/body/div[5]/div[4]/aside[1]/div[2]/div[1]/div[2]/div/form/footer/div[2]/div[1]/button").click()
                            time.sleep(4)
                            print("click")
                            driver.find_element(By.XPATH,
                                                "/html/body/div[5]/div[4]/aside[1]/div[2]/div[1]/header/div[4]/button[3]").click()
                            time.sleep(0.5)
                            print("click")
                            sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!B{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Connected"]]}).execute()

                            sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!C{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Connected one!"]]}).execute()

                            sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!D{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Message sent"]]}).execute()
                        except NoSuchElementException:
                            print("Cant Send Message")
                            sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!B{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Connected"]]}).execute()

                            sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!C{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Connected one!"]]}).execute()

                            sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!D{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Message not sent"]]}).execute()
            except NoSuchElementException as e:
                    print("Not Connected" + link['values'][0][0])
                    sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!B{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Not Connected"]]}).execute()

                    sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!C{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Not Connected"]]}).execute()

                    sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=f'Sheet1!D{row}',
                                       valueInputOption="USER_ENTERED", body = {'values': [["Message not sent"]]}).execute()
                    print("write in csv!")



In [ ]:
driver = web_driver()

Login_Linkedin(driver)
autoConnect(driver)
driver.quit()

Đã nhấn nút đăng nhập
Login không thành công
This person is not connected , try to click connect..........
click connect!
Invite sent !http://www.linkedin.com/in/jlober
This person is not connected , try to click connect..........
click connect!
Invite sent !http://www.linkedin.com/in/brett-good-5873354
This person is not connected , try to click connect..........
click connect!
Invite sent !http://www.linkedin.com/in/greg-sanossian-5187955a
This person is not connected , try to click connect..........
click connect!
Invite sent !http://www.linkedin.com/in/brianpwong
This person is not connected , try to click connect..........
click connect!
Not connected one !http://www.linkedin.com/in/owenstrevor
Pending!http://www.linkedin.com/in/zperret
write in csv!
This person is not connected , try to click connect..........
click connect!
Invite sent !http://www.linkedin.com/in/yoavgilat
This person is not connected , try to click connect..........
click connect!
Invite sent !http://www.linked